In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px grey solid !important;
  color: white !important;
}
</style>

In [2]:
import os
import re
import demoji
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# processing of text
import stanza
import spacy
from pyMorfologik import Morfologik

# models
import torch
import lazypredict
import xgboost

# setting display
pd.options.mode.chained_assignment = None
pd.options.display.max_colwidth = 300
pd.set_option("display.colheader_justify","left")
%matplotlib inline

# load and download
demoji.download_codes()
spacy_pl = spacy.load('pl_spacy_model')

... OK (Got response in 0.52 seconds)
Writing emoji data to C:\Users\patry\.demoji\codes.json ...
... OK


In [3]:
data_poleval_raw = pd.read_csv('converted_label_poleval.txt', 
                           error_bad_lines=False, sep=",")
data_github_raw = pd.read_csv('labeled_dataset.txt', 
                          error_bad_lines=False, sep="|")

data_poleval = data_poleval_raw.copy(deep=True)
data_github = data_github_raw.copy(deep=True)
datasets = [data_poleval, data_github]
conected_data = pd.concat([data_poleval, data_github.drop(['No', 'Yes'], 
                                            axis=1)], 
                          axis=0, 
                          ignore_index=True)

In [4]:
for data in datasets:
    display(data.head(20))

,Comment,Kind of offensive language
0,"Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.",0
1,@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe,0
2,"@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.",0
3,@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem",0
5,"Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD",0
6,@anonymized_account No nie ma u nas szczęścia 😉,0
7,@anonymized_account Dawno kogoś tak wrednego nie widziałam xd,0
8,"@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.",0
9,@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account,1


,Comment,No,Yes,Kind of offensive language
0,"Dziwię się, że Niemcy działają opieszale w sprawie gwałtów przecież nie wyszli chyba z wprawy z radykalnych rozwiązań tzn. w obozach dla tych islamskich zwierząt instalować przewoźne komory gazowe i po problemie.",3,28,2
1,Przywalić atomem tej islamskiej dziczy. Merkel powinna być powieszona publicznie.,2,7,2
2,"Tu trzeba jasno powiedzieć,wracasz albo kulka.",0,40,2
3,umrzesz za tydzień,16,0,2
4,"wybic do nogi prawie wszystkich arabów , pozostawiając kilku jako eksponaty",1,1,2
5,SPALIĆ MECZETY,1,1,2
6,"Czas odpalic piece i komory gazowe, szybko zboczencow muzulmanskich osądzać i w dym obracać.",1,14,2
7,Stryczek !,11,4,2
8,Bohater narodowy szkalowany przez prawicę która tylko nienawidzi! Śmierć popierającym PiS to antyludzie!,4,2,2
9,Proponuję wysłać do niego pluton specjalny z polonem.,12,6,2


In [5]:
data_github['Kind of offensive language'].value_counts().to_frame()\
    .rename_axis('Response', axis=1).style.set_properties(
        **{'color': 'white !important',
           'border': '1px grey solid !important',
           'text-align': 'center'}
        ).set_table_styles([{
            'selector': 'th',
            'props': [('border', '1px grey solid !important')]
        }])

Response,Kind of offensive language
1,2305
2,43


Due to only two response "1" and "2" we reject column 'no' and 'yes' because this columns don't give us necessary information.

In [6]:
data_github.drop(['No', 'Yes'], axis=1, inplace=True)

In [7]:
data_github.head()

,Comment,Kind of offensive language
0,"Dziwię się, że Niemcy działają opieszale w sprawie gwałtów przecież nie wyszli chyba z wprawy z radykalnych rozwiązań tzn. w obozach dla tych islamskich zwierząt instalować przewoźne komory gazowe i po problemie.",2
1,Przywalić atomem tej islamskiej dziczy. Merkel powinna być powieszona publicznie.,2
2,"Tu trzeba jasno powiedzieć,wracasz albo kulka.",2
3,umrzesz za tydzień,2
4,"wybic do nogi prawie wszystkich arabów , pozostawiając kilku jako eksponaty",2


In [8]:
def remove_quoting_comments(data):
    pattern = r'^RT.*'
    remove = data['Comment'].str.contains(pattern)
    data = data[~remove].reset_index(drop=True)
    return data

In [9]:
conected_data = remove_quoting_comments(conected_data)
conected_data

,Comment,Kind of offensive language
0,"Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.",0
1,@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe,0
2,"@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.",0
3,@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem",0
...,...,...
11730,Nie mam problemu z prześwitującą bluzką ale tu to jakoś wszystko po prostu kiepsko wygląda...mało zgrabnie.,1
11731,PIĘKNA TWARZ OSZPECONA WYSKUBANYMI WŁOSAMI. LUSTRA NIE MA? UJĘŁA SOBIE TYM 50% KOBIECOŚCI.,1
11732,"Szkoda ze miała stanik, mam nadziej że w przyszłym programie nie będzie go miała.",1
11733,"Przede wszystkim źle dobrany biustonosz a spódniczka też trochę za wąska. Wystarczy celebrytko obejrzeć się w lustrze, aby zobaczyć o czym piszę.",1


In [10]:
# function to extract emoji to dict
emoji_for_response_0 = []
emoji_for_response_1 = []
emoji_for_response_2 = []
def extraxt_emoticons(text,
                      emoji_for_response_0, 
                      emoji_for_response_1,
                      emoji_for_response_2
                     ):
    emoji = demoji.findall_list(text[0], desc=False)
    pattern = re.compile(r'[:;Xx]-?[\)\(dD](?=[\s\.]*)')
    text_emoji = pattern.findall(text[0].lower())   
    result = [*emoji, *text_emoji]
    if len(result)>0:
        if text[1] == 0:
            emoji_for_response_0.extend(result)
        elif text[1] == 1:
            emoji_for_response_1.extend(result)
        else:
            emoji_for_response_2.extend(result)
        return ' '.join(set(result)), len(result)
    else:
        return '', 0

In [11]:
conected_data['Emoticons'], conected_data['Number of emoticons in comment'] = \
    zip(*conected_data[['Comment', 'Kind of offensive language']].apply(\
                                       extraxt_emoticons, axis=1,
                                       args=(emoji_for_response_0,
                                            emoji_for_response_1,
                                            emoji_for_response_2)))

In [12]:
dict_count_emoji_0 = dict((i, emoji_for_response_0.count(i)) for i in
                          emoji_for_response_0)
dict_count_emoji_1 = dict((i, emoji_for_response_1.count(i)) for i in
                          emoji_for_response_1)
dict_count_emoji_2 = dict((i, emoji_for_response_2.count(i)) for i in
                          emoji_for_response_2)

In [13]:
conected_data[(conected_data['Kind of offensive language'] == 2) 
              & (conected_data['Number of emoticons in comment']>0)]

,Comment,Kind of offensive language,Emoticons,Number of emoticons in comment
1472,"@anonymized_account @anonymized_account Jaka tv, tacy idole😁😁😁 A tak trochę poważniej, to za tę napaść na dziewczynkę to tylko w ten pedalski ryj.",2,😁,3
1949,"@anonymized_account Zamknij niemiecki ryj,siedź w tej Brukseli i nie wracaj złodzieju!😬😬😬",2,😬,3


In [14]:
def create_dataframe_emoji_comparison():
    # create dataframe for comparison emoji
    plot_emoticons =pd.concat(
        [pd.DataFrame(dict_count_emoji_0.items(), columns=['emoji',
                                            'Response_0']).set_index('emoji'),
        pd.DataFrame(dict_count_emoji_1.items(), columns=['emoji', 
                                            'Response_1']).set_index('emoji'),
        pd.DataFrame(dict_count_emoji_2.items(), columns=['emoji', 
                                            'Response_2']).set_index('emoji')], 
        axis=1).sort_values(by=['Response_0', 'Response_1'], ascending=False)
    
    # select most frequency emoji
    plot_emoticons = plot_emoticons[~((plot_emoticons['Response_0']<10) & 
               (plot_emoticons['Response_1'].isna()) &
               (plot_emoticons['Response_2'].isna()))]
    return plot_emoticons

In [15]:
plot_emoticons = create_dataframe_emoji_comparison()
plot_emoticons

,Response_0,Response_1,Response_2
😁,773.0,58.0,3.0
:),676.0,40.0,NaN
😂,498.0,33.0,NaN
xd,183.0,5.0,NaN
😀,158.0,9.0,NaN
😉,148.0,NaN,NaN
;-),115.0,1.0,NaN
:d,85.0,3.0,NaN
;),78.0,10.0,NaN
😜,76.0,NaN,NaN
